In [11]:
import os
import pandas as pd
import numpy as np
from datetime import timedelta


In [12]:
loco_log=r"c:\Users\hbl\Desktop\automation scripts\RF issues\train no- 12415 loco- 39252 (log. 17 Jun)"
#loco_log=r'c:\Users\hbl\Desktop\automation scripts\rri2csvbot_v1\logscvrt\12339_UP _DIRECTION _BIU_COALFIELD EXPRESS _07.07.25'


Data reading

In [13]:
csvfolder=os.path.join(loco_log,"csv_files")
def read_csv_files(csvfolder,file_names):
    dataframes={}
    for file_name in file_names:
        matching_files = [f for f in os.listdir(csvfolder) if file_name in f and f.endswith(".csv")]
        if matching_files:
            file_path= os.path.join(csvfolder,matching_files[0])
            df=pd.read_csv(file_path)
            dataframes[file_name]=df
            print(f"successfully read {file_name}")
        else:
            print(f"failed to read {file_name}")
    return dataframes

file_names = ["LocoRadioRegStringLog", "LocoRadioStringLog", "StnRegLogV4", "StnRespStringLog"]    
csvfiles = read_csv_files(csvfolder, file_names) 
LocoRequest=csvfiles.get("LocoRadioStringLog")
StationResponse=csvfiles.get("StnRespStringLog")
LocoRegular= csvfiles.get("LocoRadioRegStringLog")
StationRegular= csvfiles.get("StnRegLogV4")
LocoRequest=LocoRequest[['FrameNo','TimeStamp','LocoId','Abs Location','Dir','Mode','Appr Stn','Tag-Id']]
StationResponse=StationResponse[['Type', 'Length', 'FrameNo', 'TimeStamp', 'StnId', 'Abs Lctn','Dest LocoId']]
LocoRegular=LocoRegular[['FrameNo', 'TimeStamp', 'LocoId','Abs Location','Dir', 'Emergency', 'Mode', 'Tag-Id','StPrfId']]
StationRegular = StationRegular[['FrameNo', 'TimeStamp', 'Stn-Id', 'DestLocoID', 'LastRefRFID', 'PktDir', 'Frame Offset','TrainSecType']]
loco_id = LocoRegular.loc[LocoRegular['Mode'] == "FS", 'LocoId'].iloc[0] # getting the loco-idwhere the mode is FS
def data_cleaning(df,loco_id):
    if 'LocoId' in df.columns:
        df=df[df["LocoId"]==loco_id] 
    elif 'DestLocoID' in df.columns:
        df=df[df["DestLocoID"]==loco_id]
    else:
        df=df[df["Dest LocoId"]==loco_id]
    df['TimeStamp'] = pd.to_datetime(df['TimeStamp'])
    df=df.sort_values(by="FrameNo")
    df = df.drop_duplicates(subset=["FrameNo"])
    return df    
LocoRequest=data_cleaning (LocoRequest,loco_id)
StationResponse=data_cleaning (StationResponse,loco_id)
LocoRegular=data_cleaning (LocoRegular,loco_id)
StationRegular =data_cleaning (StationRegular,loco_id)
print(StationRegular.head(6))


successfully read LocoRadioRegStringLog


C:\Users\hbl\AppData\Local\Temp\ipykernel_4952\3562149011.py:8: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(file_path)


successfully read LocoRadioStringLog


C:\Users\hbl\AppData\Local\Temp\ipykernel_4952\3562149011.py:8: DtypeWarning: Columns (29) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(file_path)


successfully read StnRegLogV4
successfully read StnRespStringLog


C:\Users\hbl\AppData\Local\Temp\ipykernel_4952\3562149011.py:33: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['TimeStamp'] = pd.to_datetime(df['TimeStamp'])
C:\Users\hbl\AppData\Local\Temp\ipykernel_4952\3562149011.py:33: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['TimeStamp'] = pd.to_datetime(df['TimeStamp'])
C:\Users\hbl\AppData\Local\Temp\ipykernel_4952\3562149011.py:33: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['TimeStamp'] = pd.to_datetime(df['TimeStamp'])
C:\Users\hbl\AppData\Local\Temp\ipykernel_4952\3562149011.py:33: UserWarning: Could not infer for

    FrameNo           TimeStamp  Stn-Id  DestLocoID  LastRefRFID PktDir  \
8     13655 2025-07-23 03:47:35   33080       39252          551      R   
9     13657 2025-07-23 03:47:37   33080       39252          551      R   
11    13659 2025-07-23 03:47:39   33080       39252          551      R   
15    13661 2025-07-23 03:47:41   33080       39252          551      R   
18    13663 2025-07-23 03:47:43   33080       39252          551      R   
21    13665 2025-07-23 03:47:45   33080       39252          551      R   

    Frame Offset TrainSecType  
8            0.0      Station  
9            0.0      Station  
11           0.0      Station  
15           0.0      Station  
18           0.0      Station  
21           0.0      Station  


C:\Users\hbl\AppData\Local\Temp\ipykernel_4952\3562149011.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['TimeStamp'] = pd.to_datetime(df['TimeStamp'])


In [14]:
print(StationRegular.columns)

Index(['FrameNo', 'TimeStamp', 'Stn-Id', 'DestLocoID', 'LastRefRFID', 'PktDir',
       'Frame Offset', 'TrainSecType'],
      dtype='object')


In [ ]:
import pandas as pd
import numpy as np

class RFWeakASPBlank:
    def __init__(self, LocoRequest, StationResponse, LocoRegular, StationRegular, loco_id, direction):
        self.direction = direction
        self.loco_id = loco_id
        self.loco_rqst = LocoRequest[LocoRequest['Dir'] == direction]
        self.stn_resp = StationResponse
        self.loco_reg = LocoRegular[LocoRegular['Dir'] == direction]
        self.stn_reg = StationRegular[StationRegular['PktDir'] == direction]
        self.unique_stations = self.loco_rqst['Appr Stn'].unique().tolist()

    def after_registration_check(self, station, reg_df,lreg_time,sreg_time,LSmt):
        nxt_stn_index=self.unique_stations.index(station)
        if nxt_stn_index < len(self.unique_stations) - 1:
            next_station_id = self.unique_stations[nxt_stn_index + 1]
            next_stn_req_time = self.loco_rqst[self.loco_rqst['Appr Stn']==next_station_id]['TimeStamp'].iloc[0]
            Aft_reg_sreg= self.stn_reg[self.stn_reg['Stn-Id']==station].copy()
            last_reg_time=Aft_reg_sreg['TimeStamp'].iloc[-1]
            print(f"curstn={last_reg_time} next stn= {next_stn_req_time} cur-nxt= {(last_reg_time-next_stn_req_time).total_seconds()}")
            if not Aft_reg_sreg.empty:
                Aft_reg_sreg['TimeDifference']=Aft_reg_sreg['TimeStamp'].diff().dt.total_seconds()
                missing=Aft_reg_sreg[Aft_reg_sreg['TimeDifference']>=4]
                print(missing)
                if missing.empty:
                    print("no stn reg packets were missing")
                elif not LSmt.empty and (LSmt['TimeStamp'] > sreg_time).any():
                    print("LS came after registration")
                elif missing.empty and not LSmt.empty and (LSmt['TimeStamp'] > sreg_time).any():
                    print("no station packets are there")
                else:
                    print("idk")
            else:#stnreg is empty
                if not LSmt.empty and (LSmt['TimeStamp'] > sreg_time).any():
                    print("there are no stnreg packets and loco went to LS mode")
                else:
                    print("After registering there are no stn reg packets in ")

    def registration_check(self, station):
        try:
            Loco_req_MT= self.loco_rqst[self.loco_rqst['Appr Stn'] == station]
            lrqst_time = Loco_req_MT['TimeStamp'].iloc[0]
            
        except IndexError:
            print(f"No loco request found for station {station}")
            return
        LSmt = Loco_req_MT[(Loco_req_MT['Mode'] == "LS") & (Loco_req_MT['Mode'].shift(1) == "FS")]
        
        
        if not LSmt.empty :
           print(f"loco switched to ls mode in {station} at {LSmt['TimeStamp']} ")
        filtered_resp = self.stn_resp[
            (self.stn_resp['StnId'] == station) & 
            (self.stn_resp['TimeStamp'] >= lrqst_time)
        ]

        sresp_time = lreg_time = sreg_time = np.nan

        if not filtered_resp.empty:
            first_resp_time = filtered_resp['TimeStamp'].iloc[0]
            if first_resp_time - lrqst_time < pd.Timedelta(minutes=5):
                sresp_time = first_resp_time

                lreg_filter = self.loco_reg[
                    (self.loco_reg['TimeStamp'] >= sresp_time) & 
                    (self.loco_reg['StPrfId'] == 0)
                ]
                if not lreg_filter.empty:
                    lreg_time = lreg_filter['TimeStamp'].iloc[0]

                    sreg_filter = self.stn_reg[
                        (self.stn_reg['TimeStamp'] >= lreg_time) & 
                        (self.stn_reg['Stn-Id'] == station)
                    ]
                    if not sreg_filter.empty:
                        sreg_time = sreg_filter['TimeStamp'].iloc[0]

        process_steps = ['Loco request', 'Station response', 'Loco regular', 'Station regular']
        timestamps = [lrqst_time, sresp_time, lreg_time, sreg_time]
        reg_df = pd.DataFrame({'registration process': process_steps, 'time': timestamps})
        reg_df['time difference'] = reg_df['time'].diff().dt.total_seconds()
        print(reg_df)
        stnregtime=reg_df[reg_df['registration process']=="Station regular"]['time'].iloc[0]
        
        #print(stnregtime)
        
        if reg_df['time'].isnull().any() :# LS mode due to loco not registered
            missing = reg_df[reg_df['time'].isnull()]['registration process'].tolist()
            if not LSmt.empty:
                print(f"=> Registration in {station} failed due to missing: {', '.join(missing)} and loco switched to LS mode at {LSmt['TimeStamp']}")
            else:
                print(f"=> Registration in {station} failed due to missing: {', '.join(missing)}")
        elif reg_df['time difference'].sum() <= 6 and not reg_df['time'].isnull().any():
            print(f"=> Loco registration with {station} station was perfect")
            self.after_registration_check(station, reg_df,lreg_time,sreg_time,LSmt)
        elif not LSmt.empty and not reg_df['time'].isnull().any() and LSmt['TimeStamp'].iloc[0]<stnregtime:# ls mode check between registration 
            first_ls_time=LSmt['TimeStamp'].iloc[0]
            delays = reg_df[
                (reg_df['time difference'] > 2) & reg_df['time difference'].notnull()
            ]
            if not delays.empty:
                delay_msgs = [
                    f"{row['registration process']} delayed by {row['time difference']} secs at {row['time']}"
                    for _, row in delays.iterrows()
                ]
                print(f"=> Registration delayed and loco switched to LS mode at {first_ls_time} in {station} station due to {', and '.join(delay_msgs)}")
            self.after_registration_check(station, reg_df,lreg_time,sreg_time,LSmt)
        else:
                
            delays = reg_df[
                (reg_df['time difference'] > 2) & reg_df['time difference'].notnull()
            ]
            if not delays.empty:
                delay_msgs = [
                    f"{row['registration process']} delayed by {row['time difference']} secs at {row['time']}"
                    for _, row in delays.iterrows()
                ]
                print(f"=> Registration delayed in {station} station due to {', and '.join(delay_msgs)}")
            self.after_registration_check(station, reg_df,lreg_time,sreg_time,LSmt)

    def process(self):
        for station in self.unique_stations:
            print(f"\nChecking station: {station}")
            self.registration_check(station)
print("************ Nominal direction analysis ************")
rf_nom = RFWeakASPBlank(LocoRequest, StationResponse, LocoRegular, StationRegular, loco_id, direction="N")
rf_nom.process()

print("************ Reverse direction analysis ****************************************************************************************")
rf_rev = RFWeakASPBlank(LocoRequest, StationResponse, LocoRegular, StationRegular, loco_id, direction="R")
rf_rev.process()

************ Nominal direction analysis ************

Checking station: 24089
  registration process                time  time difference
0         Loco request 2025-07-23 19:57:31              NaN
1     Station response 2025-07-23 19:57:33              2.0
2         Loco regular 2025-07-23 19:57:35              2.0
3      Station regular 2025-07-23 19:57:39              4.0
=> Registration delayed in 24089 station due to Station regular delayed by 4.0 secs at 2025-07-23 19:57:39
curstn=2025-07-23 20:03:05 next stn= 2025-07-23 20:02:55 cur-nxt= 10.0
       FrameNo           TimeStamp  Stn-Id  DestLocoID  LastRefRFID PktDir  \
13606    72003 2025-07-23 20:00:03   24089       39252          928      N   
13620    72023 2025-07-23 20:00:23   24089       39252          928      N   
13679    72099 2025-07-23 20:01:39   24089       39252          932      N   
13720    72155 2025-07-23 20:02:35   24089       39252          280      N   

       Frame Offset TrainSecType  TimeDifference  
13